# City Hall Data Construction

## Setup

In [2]:
import re
import json
import bisect
import requests
import datetime
from pathlib import Path

import yt_dlp
import ffmpeg
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

## Datacard

- `meetingId`
- `meetingCd`
- `meetingStartDate`
- `meetingEndDate`
- `meetingSpecialFlag`
- `meetingYear`: because sometime it is nicer to not have to use regex on meetingReference.
- `meetingUrl`: (str) A url linking to the city council meeting agenda. e.g. [https://secure.toronto.ca/council/#/committees/2462/24406](https://secure.toronto.ca/council/#/committees/2462/24406)
- `itemId`: known as agendaItemId in the City of Toronto internal meeting api.
- `itemCd`: stored as year.item code to comply with other City of Toronto data sources. A combination of nativeTermYear and referenceNumber in the City of Toronto internal meeting api.
- `itemCategory`: known as agendaSectionTitle in the City of Toronto internal meeting api.
- `itemTitle`: known as agendaItemTitle in the City of Toronto internal meeting api.
- `itemConsiderType`: known  as considerTypeCd in the City of Toronto internal meeting api.
- `itemStatus`: known as itemStatus in the City of Toronto internal meeting api.
- `itemNativeStatus`: known as nativeItemStatusCd in the City of Toronto internal meeting api.
- `itemWards`: (str) known as wards in the City of Toronto internal meeting api. Its either "All", "[ward #]" or "None".
- `itemInCameraFlag`: (bool) known as inCamera in the City of Toronto internal meeting api. True of the meeting was done *in camera*, False otherwise/
- `itemStatutoryFlag`: (bool) known as statutory in the City of Toronto internal meeting api. True if the meeting is of type statutory (but idk what that means), False otherwise/
- `itemConsiderStartTime`: (int) 0 if not considered (accepted by consensus before meeting), datetime in milliseconds otherwise
- `itemUrl`: (str) A url linking to the agenda item notes. e.g. [https://secure.toronto.ca/council/agenda-item.do?item=2024.IA20.1](https://secure.toronto.ca/council/agenda-item.do?item=2024.IA20.1).
- `itemVoteFlag`: (bool) True if the item had a vote associate with it, False otherwise
- `itemVoteType`: (str) A brief summary of the vote type. Known as Motion Type in the City of Toronto voting record API
- `itemVoteDescription`: (str) A longer description of the vote. Known as Vote Description in the City of Toronto voting record API
- `itemVoteResult`: (str) A string describing if the vote was carried, and to what extent
- `voteId`: (int) The id for the vote
- `voterName`: (str) The name of the first voting (First Last)
- `vote`: (str) The vote. "Y", "N", or "A" (Absent).

## Scraping 
### Meetings
First, we need to get a list of all city council meetings for this session. To do this, we use the city's (undocumented) api. 

The first step is to get all the completed city council meetings for the current session. The "decision body" (current city council) is 2462.

In [2]:
# Scraping
body = 2462
response = requests.get(f"https://secure.toronto.ca/council/api/multiple/meeting.json?decisionBodyId={body}")
meetings = response.json()

meeting_data = []
for meeting in meetings["Records"]:
    if meeting["mtgStatusCd"] == "COMPLETE":
        meeting_data.append(
            (
                meeting["meetingId"],
                meeting["meetingReference"], # meetingCd
                meeting["meetingDate"], # meetingStartDate
                meeting["meetingEndDate"],
                meeting["specialMeetingCd"], # meetingSpecialFlag
            )
        )

# Creating dataframe
meeting_data = pd.DataFrame(meeting_data, 
                    columns = ["meetingId", "meetingCd", "meetingStartDate", "meetingEndDate", "meetingSpecialFlag"])

# Converting meeting flag to boolean
meeting_data["meetingSpecialFlag"] = meeting_data["meetingSpecialFlag"].map(lambda x: True if x!="N" else False)

# Creating URL to get meeting data
meeting_data["meetingUrl"] = meeting_data["meetingId"].map(
    lambda x: f"https://secure.toronto.ca/council/api/individual/meeting/{x}.json")

# Creating a "year" column
meeting_data["meetingYear"] = meeting_data["meetingStartDate"].map(
    lambda dt: datetime.datetime.fromtimestamp(dt/1000).year
)

In [3]:
display(len(meeting_data))
meeting_data.head(2)

23

,meetingId,meetingCd,meetingStartDate,meetingEndDate,meetingSpecialFlag,meetingUrl,meetingYear
0,24391,2024.CC23,1731474000000,1731560400000,False,https://secure.toronto.ca/council/api/individu...,2024
1,24390,2024.CC22,1728446400000,1728532800000,False,https://secure.toronto.ca/council/api/individu...,2024


### Agenda Items

Once we have all the meetings, we need to pull out all the agenda items.

In [4]:
#meeting_data = meeting_data.sort_values("meetingStartDate", ascending=False).iloc[0:3].copy()

In [5]:
agenda_data = []
for _, r in meeting_data.iterrows(): 

    # Get meeting data
    response = requests.get(r["meetingUrl"])
    meeting = response.json()
    
    # Get agenda sections and items
    for s in meeting["Record"]["sections"]:
        for i in s["agendaItems"]:
            agenda_data.append(
                (
                    r["meetingId"],
                    i["agendaItemId"],  # itemId
                    f"{i["nativeTermYear"]}.{i["referenceNumber"]}",  # itemCd
                    s["agendaSectionTitle"],  # itemCategory
                    i["agendaItemTitle"],  # itemTitle
                    i["considerTypeCd"],  # itemConsiderType
                    i["itemStatusCd"],  # itemStatus
                    i["nativeItemStatusCd"],  # itemNativeStatus
                    i.get("wards", "None"),  # itemWards
                    i["inCamera"],  # itemInCameraFlag
                    i["statutory"],  # itemSatutoryFlag
                    i.get("considerStartTime", 0),  # itemConsiderStartTime
                    f"https://secure.toronto.ca/council/agenda-item.do?item={i["nativeTermYear"]}.{i["referenceNumber"]}" #  itemUrl
                )
            )
print(len(agenda_data[0]))

13


In [6]:
cols = [
    "meetingId",
    "itemId",
    "itemCd",
    "itemCategory",
    "itemTitle",
    "itemConsiderType",
    "itemStatus",
    "itemNativeStatus",
    "itemWards",
    "itemInCameraFlag",
    "itemStatutoryFlag",
    "itemConsiderStartTime",
    "itemUrl",
]

agenda_data = pd.DataFrame(data=agenda_data, columns=cols)

# Datatype cleaning
agenda_data["itemInCameraFlag"] = agenda_data["itemInCameraFlag"].map(lambda x: True if x!="N" else False)
agenda_data["itemStatutoryFlag"] = agenda_data["itemStatutoryFlag"].map(lambda x: True if x!="N" else False)

In [7]:
display(len(agenda_data))
agenda_data.head(2)

2957

,meetingId,itemId,itemCd,itemCategory,itemTitle,itemConsiderType,itemStatus,itemNativeStatus,itemWards,itemInCameraFlag,itemStatutoryFlag,itemConsiderStartTime,itemUrl
0,24391,136759,2024.RM23.1,Routine Matters,Call to Order,ACTION,ADOPTED,ADOPTED,All,False,False,1731508627000,https://secure.toronto.ca/council/agenda-item....
1,24391,136760,2024.RM23.2,Routine Matters,Confirmation of Minutes,ACTION,AMENDED,AMENDED,All,False,False,1731510676000,https://secure.toronto.ca/council/agenda-item....


### Votes

In [8]:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
url = base_url + "/api/3/action/package_show"
params = { "id": "members-of-toronto-city-council-voting-record"}
package = requests.get(url, params = params).json()

# To get resource data:
dfs = []
meeting_years = meeting_data["meetingYear"].unique()
for idx, resource in enumerate(package["result"]["resources"]):
    
    # Extract the years the resource covers
    match = re.search(r"(\d{4}-\d{4})", resource["name"])
    if match: 
        years = match.group(0).split("-")
        years = list(range(int(years[0]), int(years[1])))

        # Only interested in resources that cover years in our dataset
        if (
            any(y in meeting_years for y in years)
            and resource["datastore_active"]
            and resource["format"] == "CSV"
        ):
            url = base_url + "/datastore/dump/" + resource["id"]
            dfs.append(pd.read_csv(url))

if len(dfs) > 0: 
    voting_data = pd.concat(dfs, ignore_index=True)
else: 
    voting_data = None

In [9]:
# Cleaning the data
voting_data["Full Name"] = voting_data["First Name"] + " " + voting_data["Last Name"]
voting_data["Vote"] = voting_data["Vote"].map(lambda v: v[0]) # Shorten to one character

name_map = {
    "Agenda Item #": "itemCd",
    "Motion Type": "itemVoteType",
    "Vote Description": "itemVoteDescription",
    "Result": "itemVoteResult",
    "_id": "voteId",
    "Full Name": "voterName",   
    "Vote": "vote",
}
voting_data = voting_data.rename(columns=name_map)
voting_data = voting_data[name_map.values()]

In [10]:
voting_data.head(2)

,itemCd,itemVoteType,itemVoteDescription,itemVoteResult,voteId,voterName,vote
0,2023.FM1.8,Nomination of a Member,Majority required - Appoint Councillor Nunziat...,"Carried, 25-1",1,Brad Bradford,Y
1,2023.FM1.8,Nomination of a Member,Majority required - Appoint Councillor Nunziat...,"Carried, 25-1",2,Alejandra Bravo,Y


### Combining Scraped Datasets

Heirarchy is meeting > agenda items > vote

In [11]:
# First merge the voting data with the agenda items
merged = pd.merge(agenda_data, voting_data, how="left", on="itemCd")
merged["itemVoteFlag"] = merged["itemVoteType"].isna()

# Then merge with meeting data
dataset = pd.merge(meeting_data, merged, how="left", on="meetingId")

dataset = dataset[["meetingId", "meetingCd", "meetingStartDate", "meetingEndDate", "meetingSpecialFlag", "meetingYear", "meetingUrl",
                   "itemId", "itemCd", "itemCategory", "itemTitle", "itemConsiderType", "itemStatus", "itemNativeStatus",
                   "itemWards", "itemInCameraFlag", "itemStatutoryFlag", "itemConsiderStartTime", "itemUrl", "itemVoteFlag",
                   "itemVoteType", "itemVoteDescription", "itemVoteResult", "voteId", "voterName", "vote",]]


Some quick sanity checks on the data would be useful. Note that not everything in the voting records is also in the agenda items: city councillors are often on subcomittees, and make votes on issues that don't ever appear before city council.

In [12]:
# Sanity checking dataframe lengths. Not everything cons
display(len(meeting_data))
display(len(agenda_data))
display(len(voting_data))
display(len(merged))
display(len(dataset))

23

2957

24654

26734

26734

In [13]:
dataset.iloc[0]

meetingId                                                            24391
meetingCd                                                        2024.CC23
meetingStartDate                                             1731474000000
meetingEndDate                                               1731560400000
meetingSpecialFlag                                                   False
meetingYear                                                           2024
meetingUrl               https://secure.toronto.ca/council/api/individu...
itemId                                                              136759
itemCd                                                         2024.RM23.1
itemCategory                                               Routine Matters
itemTitle                                                    Call to Order
itemConsiderType                                                    ACTION
itemStatus                                                         ADOPTED
itemNativeStatus         

In [14]:
dataset.dtypes

meetingId                  int64
meetingCd                 object
meetingStartDate           int64
meetingEndDate             int64
meetingSpecialFlag          bool
meetingYear                int64
meetingUrl                object
itemId                     int64
itemCd                    object
itemCategory              object
itemTitle                 object
itemConsiderType          object
itemStatus                object
itemNativeStatus          object
itemWards                 object
itemInCameraFlag            bool
itemStatutoryFlag           bool
itemConsiderStartTime      int64
itemUrl                   object
itemVoteFlag                bool
itemVoteType              object
itemVoteDescription       object
itemVoteResult            object
voteId                   float64
voterName                 object
vote                      object
dtype: object

## Adding Video Data

Before doing anything, define some helper functions that will be useful

In [21]:
def get_video_duration(vidpath: Path): 
    """ Returns the length of a video """
    try:
        probe = ffmpeg.probe(vidpath, v="error", show_entries="format=duration")
        duration = float(probe["format"]["duration"])
        return duration
    except ffmpeg.Error as e:
        print(e.stderr.decode())
        return None

def closest_different(lst, x, mode="smaller"): 
    """ Find the closest different element in a list """
    lst = sorted(lst)

    if mode=="smaller":
        i = bisect.bisect_left(lst, x)
        return lst[i-1] if i > 0 else lst[0]
    elif mode=="bigger": 
        i = bisect.bisect_right(lst, x)
        return lst[i+1] if i < len(lst) - 1 else lst[-1]
    else: 
        raise ValueError("mode must be 'bigger' or 'smaller'")
    
def clip_video(inpath: Path, outpath: Path, start: int, end: int):
    """ Clips a video using ffmpeg """
    outpath = str(outpath)

    print(f"Clipping {inpath.stem} to {outpath} on {start}-{end}")
    try:
        ffmpeg.input(inpath, ss=start, to=end).output(outpath, c="copy").run(overwrite_output=True, capture_stdout=True, capture_stderr=True)
    except ffmpeg.Error as e: 
        print(e.stderr.decode())

def concat_videos(v1, v2, outpath: Path):
    """ Concatenates two video files using demux """

    if v1.parent != v2.parent: 
        raise ValueError("Input files must be in the same directory")
    
    tmp_file = v1.parent / "filelist.txt"
    
    with open(tmp_file, "w") as f:
        f.write(f"file '{v1.name}'\nfile '{v2.name}'\n")

    outpath = str(outpath)

    #concat video
    try:
        ffmpeg.input(tmp_file, format='concat', safe=0).output(outpath, c='copy').run(overwrite_output=True, capture_stdout=True, capture_stderr=True)
    except ffmpeg.Error as e:
        print(e.stderr.decode())

    #remove temp files
    tmp_file.unlink()   

    

### Downloading video data

First we need to extract all unique meetings. I use groupby and just keep the first elements (since the agenda items don't actually matter here), I then drop any column related to voting

In [16]:
meeting_df = dataset.copy()
meeting_df = meeting_df.groupby("meetingId", as_index=False).first()

# Drop unecessary 
meeting_df = meeting_df[[c for c in meeting_df.columns if "item" not in c]]
meeting_df = meeting_df[[c for c in meeting_df.columns if "vote" not in c]]

To make my life simpler, I will only look at the three most recent meetings

In [17]:
meeting_df = meeting_df.sort_values("meetingStartDate", ascending=False).iloc[0:3]
meeting_df

,meetingId,meetingCd,meetingStartDate,meetingEndDate,meetingSpecialFlag,meetingYear,meetingUrl
18,24391,2024.CC23,1731474000000,1731560400000,False,2024,https://secure.toronto.ca/council/api/individu...
17,24390,2024.CC22,1728446400000,1728532800000,False,2024,https://secure.toronto.ca/council/api/individu...
20,24393,2024.CC21,1725508800000,1725508800000,True,2024,https://secure.toronto.ca/council/api/individu...


Then we loop through all meetings and download the corresponding videos

In [18]:
output_dir = Path("./full_videos/")
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
for _, r in meeting_df.iterrows():
    response = requests.get(r["meetingUrl"])
    meeting_json = response.json()

    for video in meeting_json["Record"]["videoArchives"]:
        start = video["startDateTime"]
        
        video_url = video["url"]

        # Get the end time of the video
        ydl_opts = {
            'format': 'worst',
            'quiet': True,
            'no_warnings': False,
            # Extract metadata first without downloading
            'skip_download': True
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl: 
                info = ydl.extract_info(video_url, download=False)
                duration = info.get('duration')  # Duration in seconds

        end = start + duration*1000

        # Download and name the files
        ydl_opts.update({
            'skip_download': False,
            'outtmpl': f"{str(output_dir)}/{r["meetingId"]}-{start}-{end}.%(ext)s"
        })

        with yt_dlp.YoutubeDL(ydl_opts) as ydl: 
            ydl.download([video_url])
        

[download]  37.5% of    1.18GiB at   15.45MiB/s ETA 00:48   

### Splitting and labelling video data
The videos have been downloaded, but they are full videos! The tricky thing is cutting them to match the agenda items.

First, we get a dataframe of all agenda items that have consideration start times (i.e. they show up in the youtube videos).

In [18]:
subset_ids = meeting_df["meetingId"].to_list()
item_df = dataset[dataset["meetingId"].isin(subset_ids)].copy()

# We don't need any vote information,
item_df = item_df.groupby("itemId", as_index=False).first()
item_df = item_df[[c for c in item_df.columns if "vote" not in c]]

# We don't need to consider items that don't show up in the youtube videos
item_df = item_df[item_df["itemConsiderStartTime"] > 0].copy()

The next step is to loop through every meeting and cut the videos according to agenda items

In [19]:
input_dir = Path("./full_videos/")

output_dir = Path("./agenda_clips/")
output_dir.mkdir(exist_ok=True)

In [ ]:
for m_id, i_df in tqdm(item_df.groupby("meetingId"), leave=False):
    
    # Get all relevant agenda items and sort them earliest to latest
    items = list(zip(i_df["itemId"], i_df["itemCd"], i_df["itemConsiderStartTime"]))
    items.sort(key=lambda x: x[2])

    # Collect metadata of relevant videos
    vid_files = {}
    for f in input_dir.glob(f"{m_id}*.mp4"):
        metadata = f.stem.split("-") # meetingId-startTime-endTime
        vid_files[int(metadata[1])] = {
            "path": f,
            "end_time": int(metadata[2])
        }

    # loop through all items
    for j, item in enumerate(tqdm(items, leave=False)):
        print(item)

        # Get the video the agenda item starts in
        item_start = item[2]
        v1_start = closest_different(vid_files.keys(), item_start)
        v1_file = vid_files[v1_start]["path"]
        
        # Get the video the agenda item ends in
        if j == len(items)-1: # if its the last agenda item of the meeting then the end is just the end of the videos
            v2_start = sorted(vid_files.keys())[-1]
            item_end = vid_files[v2_start]["end_time"] 
        else: # otherwise the end time is the start time of the next item
            item_end = closest_different([x[2] for x in items], item[2], mode="bigger")
            v2_start = closest_different(vid_files.keys(), item_end)
        v2_file = vid_files[v2_start]["path"]

            
        # Clip (/concat) the videos as needed
        if v2_file != v1_file: # if the starting and ending files are not the same, they both need to be clipped and then combined
            v1_end = vid_files[v1_start]["end_time"]

            # Convert from datetime milliseconds to the reference frame of the video
            clip_1 = (item_start - v1_start, v1_end - v1_start)
            clip_2 = (0, item_end - v2_start)      

            clip_1 = tuple(int(x / 1000) for x in clip_1)
            clip_2 = tuple(int(x / 1000) for x in clip_2)

            # Make temporary clip files
            tmp_dir = Path("./tmp/")
            tmp_dir.mkdir(exist_ok=True)

            c1_file = tmp_dir / f"c1{v1_file.suffix}"
            c2_file = tmp_dir / f"c2{v2_file.suffix}"
            
            clip_video(v1_file, c1_file, clip_1[0], clip_1[1])
            clip_video(v2_file, c2_file, clip_2[0], clip_2[1])

            # Concatenate clips
            output_file = output_dir / f"{item[0]}-{item[1]}-{item[2]}{v1_file.suffix}"
            concat_videos(c1_file, c2_file, output_file)

            # Delete temp files
            c1_file.unlink()
            c2_file.unlink()
            
        else: # simply clip the video
            # Convert from datetime milliseconds to the reference frame of the video
            clip = (item_start - v1_start, item_end - v1_start)
            clip = tuple(int(x/1000) for x in clip)
            
            
            output_file = output_dir / f"{item[0]}-{item[1]}-{item[2]}{v1_file.suffix}"
            clip_video(v1_file, output_file, clip[0], clip[1])
            

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/77 [00:00<?, ?it/s]

(136502, '2024.RM22.1', 1728481191000)
Clipping 24390-1728480600000-1728490489000 to agenda_clips\136502-2024.RM22.1-1728481191000.mp4 on 591-2085
(136508, '2024.RM22.6', 1728481426000)
Clipping 24390-1728480600000-1728490489000 to agenda_clips\136508-2024.RM22.6-1728481426000.mp4 on 826-2120
(136503, '2024.RM22.2', 1728482685000)
Clipping 24390-1728480600000-1728490489000 to agenda_clips\136503-2024.RM22.2-1728482685000.mp4 on 2085-3061
(136505, '2024.RM22.3', 1728482720000)
Clipping 24390-1728480600000-1728490489000 to agenda_clips\136505-2024.RM22.3-1728482720000.mp4 on 2120-3077
(136506, '2024.RM22.4', 1728483661000)
Clipping 24390-1728480600000-1728490489000 to agenda_clips\136506-2024.RM22.4-1728483661000.mp4 on 3061-3089
(136507, '2024.RM22.5', 1728483677000)
Clipping 24390-1728480600000-1728490489000 to agenda_clips\136507-2024.RM22.5-1728483677000.mp4 on 3077-4180
(136509, '2024.RM22.7', 1728483689000)
Clipping 24390-1728480600000-1728490489000 to tmp\c1.mp4 on 3089-9889
Clipp

### Checking clip lengths

In [6]:
for file in Path("./agenda_clips/").glob("*.mp4"): 
    print(file)
    probe = ffmpeg.probe(file)

    duration = float(probe['format']['duration']) / 60

    print(f"Video duration: {duration} seconds")

agenda_clips\136118-2024.EC14.11-1728507776000.mp4


FileNotFoundError: [WinError 2] The system cannot find the file specified